In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer

# Huawei dashboard file
main_file = '/content/drive/MyDrive/Python Data Base/GUL WCL Analysis/(WK35_Q2-2023) Huawei GUL CS & PS WCL Tracker_2023-09-08.xlsx'

# Output file link
output_file = '/content/drive/MyDrive/Python Data Base/GUL WCL Analysis'

t0 = timer()

# Week numbers to calculate
weeks = ['WK 30', 'WK 31', 'WK 32', 'WK 33', 'WK 34', 'WK 35']

# GSM related KPIs
g_kpi = ['CS CSSR (%)', 'HOSR (%)', 'TCH DR (%)', 'TCH Cong (%)', 'SDCCH Cong (%)']
g_extra = ['CS CSSR Denum', 'HOSR Denum', 'TCH DR Denum', 'TCH Cong Num', 'SDCCH Cong Num']
g_kpi_crit = [90, 90, 5, 1, 1]
g_extra_crit = [100, 100, 100, 20, 20]
g_kpi_type = ['sr', 'sr', 'dr', 'dr', 'dr']
gmc = ['Mute Call Ratio       (BBH, >=3%)', 'Mute Call Num      (Daily, >10 Call)']

# LTE related KPIs
l_kpi = ['E2E CSSR (%)', 'SDR (%)', 'CSFB Prep SR (%)', 'Intra Freq HOSR (%)', 'Inter-RAT HOSR %','PRB Utilization ']
l_extra = ['E2E CSSR Denum ', np.nan, 'CSFB Prep SR Denum', 'Intra Freq HOSR Denum', 'Inter-RAT HOSR % Denum', np.nan]
l_kpi_crit = [96, 1, 97, 98, 95, 90]
l_extra_crit = [100, np.nan, 100, 100, 50, np.nan]
l_kpi_type = ['sr', 'dr', 'sr', 'sr', 'sr', 'dr']

# UMTS related KPIs
u_kpi = ['CS CSSR (%)', 'PS CSSR (%)', 'CS CDR (%)', 'PS CDR (%)', 'Soft HOSR (%)','CS Inter-RAT HOSR (%)',
         'DL Power congestion count', 'Code congestion count', 'TCP Utilization']
u_extra = ['CS CSSR Denum', 'PS CSSR Denum', 'CS CDR Denum', 'PS CDR Denum', 'Soft HO SR Denum',
           'CS Inter-RAT HOSR Denum', np.nan, np.nan, np.nan]
u_kpi_crit = [95, 95, 3, 3, 98, 90, 10, 10, 80]
u_extra_crit = [100, 100, 100, 100, 100, 100, np.nan, np.nan, np.nan]
u_kpi_type = ['sr', 'sr', 'dr', 'dr', 'sr', 'sr', 'sp', 'sp', 'sp']
umc = ['Mute Call Ratio       (BBH, >=3%)', 'Mute Call Num      (Daily,  >10 Call)']

# GSM data loading & wrangling
print('Loading GSM data...')
G = pd.read_excel(main_file, sheet_name = 'GSM-KPIs', header=[0,1], index_col=0, na_values=['#DIV/0!', '#VALUE!'])
G.reset_index(drop=True, inplace=True)
G.drop(index=0, axis=0, inplace=True)
G_MC = pd.read_excel(main_file, sheet_name = 'GSM-Mute Call', header=[0,1], index_col=0, na_values=['#DIV/0!', '#VALUE!'])
g_frame = pd.DataFrame()

# GSM processing
print('GSM processing...')
# For main KPIs
for kpi, extra, k_crit, ex_crit, type_ in zip(g_kpi, g_extra, g_kpi_crit, g_extra_crit, g_kpi_type):
    for week in weeks:
        if type_ == 'sr':
            new_cells = G[(G[week][kpi] < k_crit) & (G[week][extra] > ex_crit)] ['Unnamed: 1_level_0']['Cell Name']
        else:
            new_cells = G[(G[week][kpi] >= k_crit) & (G[week][extra] >= ex_crit)] ['Unnamed: 1_level_0']['Cell Name']

        g_frame = g_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
                                           'Week': week}).reset_index(drop=True), ignore_index=True)
        g_frame.drop_duplicates(subset=['Cell Name', 'KPI'], keep='first', ignore_index=True, inplace=True)

# For Mute Call
for week in weeks:
    new_cells = G_MC[(G_MC[week][gmc[0]] >= 3) & (G_MC[week][gmc[1]] > 10)] ['Unnamed: 1_level_0']['Cell Name']
    g_frame = g_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': 'Mute Call',
                                       'Week': week}).reset_index(drop=True), ignore_index=True)
    g_frame.drop_duplicates(subset=['Cell Name', 'KPI'], keep='first', ignore_index=True, inplace=True)
del G, G_MC

# LTE processing
# LTE data loading & wrangling
print('Loading LTE data...')
L = pd.read_excel(main_file, sheet_name = 'LTE-KPIs', header=[0,1], index_col=0, na_values=['#DIV/0!', '#VALUE!'])
L.reset_index(drop=True, inplace=True)
L.drop(index=0, axis=0, inplace=True)
l_frame = pd.DataFrame()

# For main KPIs
print('LTE processing...')
for kpi, extra, k_crit, ex_crit, type_ in zip(l_kpi, l_extra, l_kpi_crit, l_extra_crit, l_kpi_type):
    for week in weeks:
        if type_ == 'sr':
            new_cells = L[(L[week][kpi] < k_crit) & (L[week][extra] > ex_crit)] ['Unnamed: 1_level_0']['Cell Name']
        else:
            new_cells = L[(L[week][kpi] >= k_crit)] ['Unnamed: 1_level_0']['Cell Name']

        l_frame = l_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
                                           'Week': week}).reset_index(drop=True), ignore_index=True)
        l_frame.drop_duplicates(subset=['Cell Name', 'KPI'], keep='first', ignore_index=True, inplace=True)
del L

# UMTS processing
# UMTS data loading & wrangling
print('Loading UMTS data...')
U = pd.read_excel(main_file, sheet_name = 'UMTS-KPIs', header=[0,1], index_col=0, na_values=['#DIV/0!', '#VALUE!'])
U.reset_index(drop=True, inplace=True)
U.drop(index=0, axis=0, inplace=True)
U_MC = pd.read_excel(main_file, sheet_name = 'UMTS-Mute Call', header=[0,1], index_col=0, na_values=['#DIV/0!', '#VALUE!'])
u_frame = pd.DataFrame()

# For main KPIs
print('UMTS processing...')
for kpi, extra, k_crit, ex_crit, type_ in zip(u_kpi, u_extra, u_kpi_crit, u_extra_crit, u_kpi_type):
    for week in weeks:
        if type_ == 'sr':
            new_cells = U[(U[week][kpi] < k_crit) & (U[week][extra] > ex_crit)] ['Unnamed: 1_level_0']['Cell Name']
        elif type_ == 'dr':
            new_cells = U[(U[week][kpi] >= k_crit) & (U[week][extra] > ex_crit)] ['Unnamed: 1_level_0']['Cell Name']
        else:
            U[(U[week][kpi] >= k_crit)] ['Unnamed: 1_level_0']['Cell Name']

        u_frame = u_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
                                           'Week': week}).reset_index(drop=True), ignore_index=True)
        u_frame.drop_duplicates(subset=['Cell Name', 'KPI'], keep='first', ignore_index=True, inplace=True)

    # For Mute Call
for week in weeks:
    new_cells = U_MC[(U_MC[week][umc[0]] >= 3) & (U_MC[week][umc[1]] > 10)] ['Unnamed: 1_level_0']['Cell Name']
    u_frame = u_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': 'Mute Call',
                                       'Week': week}).reset_index(drop=True), ignore_index=True)
    u_frame.drop_duplicates(subset=['Cell Name', 'KPI'], keep='first', ignore_index=True, inplace=True)
del U, U_MC

print('Exporting....')
with pd.ExcelWriter(output_file+'/GUL WCL Dashboard Cell List by HAL.xlsx') as writer:
    g_frame.to_excel(writer, sheet_name = 'GSM', index=False)
    u_frame.to_excel(writer, sheet_name = 'UMTS', index=False)
    l_frame.to_excel(writer, sheet_name = 'LTE', index=False)

et = timer()

print('Operation completed with time consumption:', (et-t0)/60, 'minutes.')

Loading GSM data...
GSM processing...


<ipython-input-42-6fd04851571e>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  g_frame = g_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6fd04851571e>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  g_frame = g_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6fd04851571e>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  g_frame = g_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6fd04851571e>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  g_frame = g_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6f

Loading LTE data...
LTE processing...


<ipython-input-42-6fd04851571e>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  l_frame = l_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6fd04851571e>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  l_frame = l_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6fd04851571e>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  l_frame = l_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6fd04851571e>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  l_frame = l_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6f

Loading UMTS data...
UMTS processing...


<ipython-input-42-6fd04851571e>:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  u_frame = u_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6fd04851571e>:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  u_frame = u_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6fd04851571e>:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  u_frame = u_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-42-6fd04851571e>:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  u_frame = u_frame.append(pd.DataFrame({'Cell Name':new_cells, 'KPI': kpi,
<ipython-input-4

Exporting....
Operation completed with time consumption: 2.748570161233329 minutes.
